<a href="https://colab.research.google.com/github/Tendo15/MscTimeSeries/blob/main/Finland_Helsinki_TIME_SERIES_Pressure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Exploratory Data Analysis

In [ ]:
weather_data = pd.read_csv('/content/drive/MyDrive/ColabNotebooks2024/Disso24/finland_data_test.csv',parse_dates=['datetime'],sep=';', decimal=',', infer_datetime_format=True)
#Check the shape of the dataset
print(weather_data.shape)

#Select the datetime and the Atmospheric Pressure(reduced to mean sea level) columns
helsinki_df=weather_data[["datetime","P_mu"]]
helsinki_df.head(10)

# DATA SELECTION FROM 2016 TO 2019

In [ ]:
#Select the subset data from 2016 to 2019
mask = (helsinki_df['datetime'] >= '2016-01-01') & (helsinki_df['datetime'] <= '2019-05-21')
helsinki_df = helsinki_df.loc[mask]

#Reset the index
helsinki_df.set_index("datetime", inplace=True)

#Inspect first 5 rows and last 5 rows of the data
from IPython.display import display
display(helsinki_df.head(5))
display(helsinki_df.tail(5))

In [ ]:
helsinki_df.describe()

In [ ]:
#Output the maximum and minimum Atmospheric pressure date
print(helsinki_df.loc[helsinki_df["P_mu"] == helsinki_df["P_mu"].max()])
print(helsinki_df.loc[helsinki_df["P_mu"] == helsinki_df["P_mu"].min()])

# Observations

*   On January 2019 the lowest Atmospheric Pressure occured at 730.2  this was the lowest pressure to date in a period of 4 years
*   In july 2018 the highest Atmospheric Pressure occured at 785.9 at.
Deviation of pressure from the average pressure is associated with cyclonic / anticyclonic activity. In a cyclone pressure decreases and in an anticyclone it increases.
# SEASONS IN FINLAND
*   Summer June and lasts until the end of August
*   Autumn August and the start of September
*   Winter November and lasts until the end of March or April.
*   Spring March, but it won’t get warmer until April.

The average pressure in any location is very dependent on height of the ground relative to the sea level. With increasing altitude the pressure decreases. For example, in case of the same state of the atmosphere, the atmospheric pressure in Moscow and St. Petersburg will differ due to the fact that these cities are located at different altitudes. In Moscow, at an altitude of about 156 m above the sea level, the pressure will be approximately by 15 mmHg. lower than in St. Petersburg, located at a height of about 4 m above the sea level.

https://www.edunation.co/blog/four-seasons-finland/#:~:text=When%20students%20come%20to%20Finland,spring%2C%20summer%2C%20and%20autumn.






# Data Visualizations

In [ ]:
#plot the daily temperature change
plt.figure(figsize=(16,10), dpi=100)
plt.plot(helsinki_df.index, helsinki_df.P_mu, color='tab:red')
plt.gca().set(title="Daily Atmospheric Pressure in Helsinki International airport Vantaa Finland from 2016 to 2019", xlabel='Date', ylabel="Atmospheric Pressure (in milimeters mercury)")
plt.show()

# Moving average smoothing
This is a time series forcasting method used to remove the fine-grained variation between time stemps

*   Smoothing is used to remove noise and better expose undelying process
*   Moving avarages are a common type of smoothing in tim series forcasing

This rolling() function will group our observations into a window. This is a window size of 30 days, chosen randomly



In [ ]:
#Applying the moving Average function by a subset of size being 30 days
helsinki_df_mean=helsinki_df.P_mu.rolling(window=30).mean()
helsinki_df_mean.plot(figsize=(20,15))

Observation: a seasonality trend/ pattern is happening in this dataset whereby the temperatures are always low at the begining of the year and they increases in the middle of the year.  Usually in the month of June finland has an increase of the sun.

# ADDITIVE DECOMPOSITION

Time series decomposition allows us to to describe the trend and seasonal factors in a time series. More extensive decompositions might also include long-run cycles, holiday effects, day of week effects and so on. Here, we'll only consider trend and seasonal decompositions.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Additive Decomposition with adjusted period
result_add = seasonal_decompose(helsinki_df.P_mu, model='additive', extrapolate_trend='freq', period=365)

# Plot
plt.rcParams.update({'figure.figsize':(10,10)})
result_add.plot().suptitle('Additive Decomposition', fontsize=22)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Additive Decomposition with adjusted period
result_add = seasonal_decompose(helsinki_df.P_mu, model='additive', extrapolate_trend='freq', period=365)

# Plot with pink color for trend
plt.rcParams.update({'figure.figsize':(10,10)})
result_add.trend.plot(color='purple')
plt.title('Additive Decomposition - Trend', fontsize=22)
plt.show()

# Plot with pink color for seasonal component
plt.rcParams.update({'figure.figsize':(10,10)})
result_add.seasonal.plot(color='purple')
plt.title('Additive Decomposition - Seasonal', fontsize=22)
plt.show()

# Plot with pink color for residual
plt.rcParams.update({'figure.figsize':(10,10)})
result_add.resid.plot(color='purple')
plt.title('Additive Decomposition - Residual', fontsize=22)
plt.show()

# Baseline Model using 1-step prediction to model temp

In [ ]:
#Shift the current temperature to the next day
predicted_df = helsinki_df["P_mu"].to_frame().shift(1).rename(columns = {"P_mu": "P_mu_pred"})
actual_df = helsinki_df["P_mu"].to_frame().rename(columns={"P_mu": "P_mu_actual"})

#Concantinate the actual and predicted temp
one_step_df = pd.concat([actual_df, predicted_df],axis=1)

#Select from the second row, due to no prediction due to shifting
one_step_df = one_step_df[1:]
one_step_df.head(10)

# Root Mean Squared Error(RMSE) bettween predicted and actual
# RMSE = sqrt [(Σ(Pi – Oi)²) / n]



In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Calculate the MSE
mse = mean_squared_error(one_step_df.P_mu_actual, one_step_df.P_mu_pred)

# Calculate the RMSE
rmse = sqrt(mse)
print("The RMSE is", rmse)

- RMSE is higher than the RMSE of the tempreture Model.
- Unfortunately, based on a rule of thumb, values between 0.2 and 0.5 show model acuracy.

# FORECASTING USING THE SARIMA MODEL

SARIMA: Seasonal Autoregressive, Integrated Moving Average.
SARIMA Notation: (p,d,q)(P,D,Q,s)
these parameters = Seasonality, Trend and noise.

# PARAMETER SELECTION USING GRID SEARCH
*   Grid search used to explore different combinations of parameters.
*   For each combination of parameters, we fit new seasonal SARIMA model using the SARIMAX()function



In [ ]:
import itertools

#Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)

#Create all different combinations of p, d and q triple
pdq = list(itertools.product(p, d, q))

#Create all diffrent combinations of seasonal p,d,q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p,d,q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

# Parameter Selection using grid search

In [ ]:
import warnings
warnings.filterwarnings("ignore") #specify to ignore warning messages

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(one_step_df.P_mu_actual,
                                          order=param,
                                          seasonal_order=param_season,
                                          enforce_stationarity=False,
                                          enforce_invertibility=False)
            results = mod.fit()

            print('SARIM{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
#Import the statsmodels library for using SARIMAX model
import statsmodels.api as sm

#Fit the SARIMAX model using optimal parameters
mod = sm.tsa.statespace.SARIMAX(one_step_df.P_mu_actual,
                                order=(1,1,1),
                                seasonal_order=(1,1,1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

# Model Diagnostics - unusual behavior

# **RESULTS shows**


1.   Model resedual are normally distribuated
2.   KDE line follows closely for a normal disribution and a mean of 0 and standard deviation of 1
3. Normal corelation on QQ Plot
4. Normal distribution of 0-mean




In [ ]:
results.plot_diagnostics(figsize=(15,12))
plt.show()

# **FORECASTS VALIDATION**

To help understand th accuracy of forecasts we set forecasts to start from 2017 to the end of the data

In [ ]:
#The 'get_prediction()' and 'conf_int()' attributes allow us to obtain the values
pred = results.get_prediction(start=pd.to_datetime('2017-05-19'), dynamic=False)
pred_ci = pred.conf_int()

In [ ]:
ax = one_step_df.P_mu_actual['2015':].plot(label='Actual')
pred.predicted_mean.plot(ax=ax, label='Forecast')

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Atmospheric Pressure (in milimeters mercury)')
plt.ylim([-20,30])
plt.legend()
plt.show()

# Mean Square Error of the 5.11 forecast *MSE = Σ(yi − pi)2n*

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
from math import sqrt

y_forecasted = pred.predicted_mean
y_truth = one_step_df.P_mu_actual['2017-05-19':]
print(y_forecasted.shape)
print(y_truth.shape)
#Compute the mean square error
mse = sqrt(MSE(y_truth, y_forecasted).mean())
print('The Mean Squared Error of this forecast is{}'.format(round(mse, 2)))

# Observation:
# MSE = Σ(yi − pi)2n
Mean squared error(MSE) of the 5.11 forecast is 4.97
The closer the MSE is to the value of "0" the more perfect the accuracy of the parameter. However, 4.97 is very high

# DYNAMIC FORECASTS

- Check for a better representation of true Prediction power
- Information used is from the time series up to a specific forecast point, then after the forecasts are generated from previous values
- Cpmputing of dynamoc forecasting  and confident intervals from May 2017 onwards

In [ ]:
pred_dynamic = results.get_prediction(start=pd.to_datetime('2017-05-19'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()

In [ ]:
ax = one_step_df.P_mu_actual['2015':].plot(label='Actual', figsize=(15, 10))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], color='k', alpha=.25)

ax.set_xlabel('Date')
ax.set_ylabel('Atmospheric Pressure (milimeters of mercury)')
plt.title('Dynamic Forecats-Actual vs Observed ', fontsize=22)
plt.ylim([-20,30])
plt.legend()
plt.show()